Notebook to generate embeddings from red wine product information from Systembolaget in Swedish

In [22]:
import re
import os
import pandas as pd
import tiktoken
import openai
from openai.embeddings_utils import get_embedding

In [23]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [24]:
# load data, come cleanign and create combined row
input_datapath = "red_wines_filtered.csv"
df = pd.read_csv(input_datapath, header=0)

df.fillna("", inplace=True)
df['fruitacid'] = df['fruitacid'].astype(str)
df['body'] = df['body'].astype(str)
df['roughness'] = df['roughness'].astype(str)

# removing 'Serveras vid xx-yy C'
pattern = r"\svid\s[0-9]{1,2}-[0-9]{1,2}°C"
df['usage'] = df['usage'].apply(lambda s: re.sub(pattern, '', s))

df["combined"] = (
    "Land: " + df.country.str.strip() + \
    "; Druvor: " + df.grapes.str.strip() + \
    "; Smak: " + df.category.str.strip() + ". " + df.taste.str.strip() + \
    "; Servering: " + df.usage.str.strip() + \
    "; Fruktsyra: " + df.fruitacid.str.strip() + " (0-12)" + \
    "; Fyllighet: " + df.body.str.strip() + " (0-12)" + \
    "; Strävhet: " + df.roughness.str.strip() + " (0-12)"
)

print(df["combined"][0])

Land: Italien; Druvor: Primitivo, Malvasía, Negroamaro; Smak: Kryddigt & Mustigt. Kryddig smak med fatkaraktär, inslag av mörka körsbär, choklad, romrussin, peppar och vanilj.; Servering: Serveras till rätter av lamm- eller nötkött, gärna grillat.; Fruktsyra: 9 (0-12); Fyllighet: 9 (0-12); Strävhet: 8 (0-12)


In [25]:
# calculate tokens tiktoken
encoding = tiktoken.get_encoding(embedding_encoding)
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
print(df["n_tokens"].sum())

21278


In [51]:
# test embeddings with cosine similarity
import numpy as np
from openai.embeddings_utils import cosine_similarity

datafile_path = "red_wines_filtered_with_embeddings.csv"
df_with_embeddings = pd.read_csv(datafile_path)
df_with_embeddings["embedding"] = df_with_embeddings.embedding.apply(eval).apply(np.array)

# use first row as test case
first_row = df_with_embeddings.iloc[0]
first_row_embedding = first_row.embedding


# every row except first
df_hold_out_first = df_with_embeddings.tail(-1)

df_hold_out_first["similarity"] = df_hold_out_first.embedding.apply(lambda x: cosine_similarity(x, first_row_embedding))

results = (
        df_hold_out_first.sort_values("similarity", ascending=False)
        .head(5)["combined"])

Land: Italien; Druvor: Primitivo, Malvasía, Negroamaro; Smak: Kryddigt & Mustigt. Kryddig smak med fatkaraktär, inslag av mörka körsbär, choklad, romrussin, peppar och vanilj.; Servering: Serveras till rätter av lamm- eller nötkött, gärna grillat.; Fruktsyra: 9 (0-12); Fyllighet: 9 (0-12); Strävhet: 8 (0-12)

5 MOST SIMILAR
Land: Italien; Druvor: Primitivo; Smak: Kryddigt & Mustigt. Kryddigt, smakrikt vin med liten sötma, inslag av fat, romrussin, choklad, vanilj, mörka körsbär, plommon och nötter.; Servering: Serveras till rätter av lamm- eller nötkött.; Fruktsyra: 9 (0-12); Fyllighet: 9 (0-12); Strävhet: 8 (0-12)

Land: Italien; Druvor: Primitivo; Smak: Fruktigt & Smakrikt. Generöst fruktig smak med fatkaraktär och liten sötma, inslag av mörka körsbär, choklad, russin, kaffe, sötlakrits och vanilj.; Servering: Serveras till grillade rätter av lamm- eller nötkött, eller till lagrade hårdostar.; Fruktsyra: 8 (0-12); Fyllighet: 7 (0-12); Strävhet: 6 (0-12)

Land: Italien; Druvor: Primit

/var/folders/_h/k54qpypj7ds3ds3v29bxr6mm0000gn/T/ipykernel_57198/4217679176.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hold_out_first["similarity"] = df_hold_out_first.embedding.apply(lambda x: cosine_similarity(x, first_row_embedding))
